# TG Bot load and test the messages

Package name=>python-telegram-bot is not installed on your python environment, either install it ( better to install requirements.txt altogether) or turn on environment variable EXHORT_PYTHON_VIRTUAL_ENV=true to automatically installs it on virtual environment ( will slow down the analysis)

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

# load the API key from the environment
TOKEN = os.getenv("TOKEN")

# print(TOKEN)

Persistent Data

In [2]:
import sqlite3

# Replying to /start

In [3]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import logging

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)


def start(update, context):
    logger.info('User %s started the conversation.', update.message.from_user.first_name)
    update.message.reply_text('Hello! This is your bot. Made with 🩷 by @AdityaPanwars')

In [4]:
def save_note(update, context, cursor, conn):
    # Check if the reply is a text message
    if update.message.reply_to_message and update.message.reply_to_message.text:
        # Get the text of the replied message
        note_text = update.message.reply_to_message.text

        # Get the note name from the command
        command_parts = update.message.text.split()
        if len(command_parts) >= 2:
            note_name = command_parts[1]

            # Save the note to SQLite database
            cursor.execute('INSERT OR REPLACE INTO notes (note_name, note_text) VALUES (?, ?)', (note_name, note_text))
            conn.commit()
            update.message.reply_text(f'Note "{note_name}" saved successfully.')
        else:
            update.message.reply_text('Please provide a note name.')
    else:
        update.message.reply_text('Please reply to a text message to save a note.')

    logger.info('User %s is saving a note.', update.message.from_user.first_name)


In [5]:
def list_notes(update, context, cursor, conn):
    # check if the user passes the prefix
    if len(context.args) > 0:
        prefix = context.args[0]
        cursor.execute('SELECT note_name FROM notes WHERE note_name LIKE ? ORDER BY note_name ASC', (prefix + '%',))
    else:
        cursor.execute('SELECT note_name FROM notes ORDER BY note_name ASC')
    
    result = cursor.fetchall()

    if result:
        notes_list = "\n".join([f"#{note[0]}" for note in result])
        update.message.reply_text(f"List of saved notes:\n{notes_list}")
    else:
        update.message.reply_text("No notes saved yet.")

    logger.info('User %s is listing the notes.', update.message.from_user.first_name)


In [6]:
def retrieve_note(update, context, cursor, conn):
    # Get the note name from the command
    note_name = update.message.text.split('#')[1].strip()

    cursor.execute('SELECT note_text FROM notes WHERE note_name = ?', (note_name,))
    result = cursor.fetchone()

    logger.info('User %s is retrieving a note.', update.message.from_user.first_name)

    if result:
        note_text = result[0]
        update.message.reply_text(f'Note "{note_name}":\n***********\n\n{note_text}')
    else:
        update.message.reply_text(f'Note "{note_name}" not found.')


In [7]:
def main():
    updater = Updater(TOKEN, use_context=True)
    dp = updater.dispatcher
    
    # Connect to SQLite database
    conn = sqlite3.connect('bot_data.db', check_same_thread=False)
    cursor = conn.cursor()

    # Create table if not exists
    cursor.execute('''CREATE TABLE IF NOT EXISTS notes (
                       note_name TEXT PRIMARY KEY,
                       note_text TEXT
                   )''')
    conn.commit()

    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("save", lambda update, context: save_note(update, context, cursor, conn)))
    dp.add_handler(MessageHandler(Filters.regex(r'#\w+'), lambda update, context: retrieve_note(update, context, cursor, conn)))
    dp.add_handler(CommandHandler("list", lambda update, context: list_notes(update, context, cursor, conn)))
    
    updater.start_polling()
    updater.idle()


In [8]:
if __name__ == '__main__':
    main()

2024-02-10 07:43:25,939 - apscheduler.scheduler - INFO - Scheduler started
2024-02-10 07:43:33,688 - __main__ - INFO - User ᗩᕲᓰᖶᖻᗩ is listing the notes.
2024-02-10 07:43:37,460 - __main__ - INFO - User ᗩᕲᓰᖶᖻᗩ is listing the notes.
2024-02-10 07:44:09,352 - __main__ - INFO - User ᗩᕲᓰᖶᖻᗩ is saving a note.
2024-02-10 07:44:29,466 - __main__ - INFO - User ᗩᕲᓰᖶᖻᗩ is saving a note.
2024-02-10 07:44:33,094 - __main__ - INFO - User ᗩᕲᓰᖶᖻᗩ is listing the notes.
2024-02-10 07:44:38,976 - __main__ - INFO - User ᗩᕲᓰᖶᖻᗩ is listing the notes.
2024-02-10 07:44:43,411 - __main__ - INFO - User ᗩᕲᓰᖶᖻᗩ is listing the notes.
